In [14]:
#Para obtener el csv de las estaciones de carga de EE.UU. con la información que necesitamos
import pandas as pd
import numpy as np

df = pd.read_csv('eeuu_fuel_stations.csv', low_memory=False)
df = df.sample(frac=1, random_state=np.random.RandomState())

df2 = df[[ "Station Name", "Latitude", "Longitude", "EV Level2 EVSE Num"]]
#Eliminamos los NULL
df2 = df2[df["EV Level2 EVSE Num"] > 0]
df2["Station Name"].astype(str)
#df2.to_csv('eeuu_fuel_stations_processed.csv', index=False)
df2


,Station Name,Latitude,Longitude,EV Level2 EVSE Num
1848,Central Illinois Regional Airport,40.484088,-88.914584,2.0
51422,Hyatt Place Fort Lee,40.862299,-73.976057,4.0
26925,LEGACY AFC STATION 2,33.847361,-84.366858,2.0
19732,CNTYOFRIVERSIDE CABAZON FLEET,33.916607,-116.785522,2.0
35282,SCI CHARGERS WEST G VALET 1,36.156190,-115.335030,2.0
...,...,...,...,...
49672,Antelope Valley Mall - South Parking,34.603272,-118.153064,2.0
27047,ATRIUM EVCHARGE RSPJ EV 3,42.868362,-73.781610,2.0
8990,Parkright 300 E 77th St - Tesla Destination,40.771794,-73.955624,3.0
24428,"Walgreens - Escondido, CA #5455",33.126395,-117.084463,1.0


In [2]:
df2["index"] = df2.index + 1
df2.head()

,Station Name,Latitude,Longitude,EV Level2 EVSE Num,index
54650,SUMMIT TOYOTA STATION 1,41.063476,-81.577133,1.0,54651
23511,Two Wisconsin Circle - Tesla Destination,38.961970,-77.085332,1.0,23512
41484,Middle Georgia State University - Cochran,32.380615,-83.346060,2.0,41485
49427,DeForest Athletic Complex,43.221461,-89.362470,4.0,49428
3059,STOWE ELECTRIC REC PATH,44.466002,-72.684932,2.0,3060


In [15]:
#Para importar la información a DynamoDB

#Importacion de las electrolineras
import boto3
from decimal import Decimal
import json

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
electrolineras_tabla = dynamodb.Table('Electrolinera')

#Eliminamos las entradas anteriores
response = electrolineras_tabla.scan()
for i in response['Items']:
    electrolineras_tabla.delete_item(
        Key={
            'ID_Electrolinera': i['ID_Electrolinera']
        }
    )

def import_electrolinera(row):
    item = {
            'ID_Electrolinera': row["index"],
            'nombre': row['Station Name'],
            'latitud': row['Latitude'],
            'longitud': row['Longitude'],
            'estaciones': row['EV Level2 EVSE Num'],
            'observaciones': "",
            'estado': 1
        }

    electrolineras_tabla.put_item(
        Item = json.loads(json.dumps(item), parse_float=Decimal)
    )

df = pd.read_csv('eeuu_fuel_stations_processed.csv', low_memory=False)
df["index"] = df.index + 1
df_mini = df.head(10)
df_mini.apply(import_electrolinera, axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

In [16]:
#Importacion de las electrolineras
import boto3
from decimal import Decimal
import json

df = pd.read_csv('eeuu_fuel_stations_processed.csv', low_memory=False)
df["index"] = df.index + 1
df_mini = df.head(10)

In [17]:
#Ahora generamos un punto de carga por cada electrolinera importada
df_mini["puntos_de_carga"] = df_mini["EV Level2 EVSE Num"].apply(lambda x: [i for i in range(1, int(x)+1)])
df_mini = df_mini.explode("puntos_de_carga")
df_mini


/tmp/ipykernel_8420/1887988630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mini["puntos_de_carga"] = df_mini["EV Level2 EVSE Num"].apply(lambda x: [i for i in range(1, int(x)+1)])


,Station Name,Latitude,Longitude,EV Level2 EVSE Num,index,puntos_de_carga
0,DELTA AIR LINES TOC EV CHARGER1,33.641652,-84.415299,2.0,1,1
0,DELTA AIR LINES TOC EV CHARGER1,33.641652,-84.415299,2.0,1,2
1,275 west Hunt Club Rd - 2,45.340513,-75.709017,4.0,2,1
1,275 west Hunt Club Rd - 2,45.340513,-75.709017,4.0,2,2
1,275 west Hunt Club Rd - 2,45.340513,-75.709017,4.0,2,3
1,275 west Hunt Club Rd - 2,45.340513,-75.709017,4.0,2,4
2,City of Pasadena - Marengo Garage,34.144703,-118.146957,5.0,3,1
2,City of Pasadena - Marengo Garage,34.144703,-118.146957,5.0,3,2
2,City of Pasadena - Marengo Garage,34.144703,-118.146957,5.0,3,3
2,City of Pasadena - Marengo Garage,34.144703,-118.146957,5.0,3,4


In [18]:

#Importacion de los puntos de carga
puntos_de_carga_tabla = dynamodb.Table('PuntoCarga')

#Eliminamos las entradas anteriores
response = puntos_de_carga_tabla.scan()
for i in response['Items']:
    puntos_de_carga_tabla.delete_item(
        Key={
            'ID_PuntoCarga': i['ID_PuntoCarga']
        }
    )

def import_punto_de_carga(row):
        item = {
                'ID_PuntoCarga': row["index"]*1000 + row["puntos_de_carga"],
                'ID_Electrolinera': row["index"],
                'num_pc_electrolinera': row['puntos_de_carga'],
                'estado': 1,
                'uptime': 0
            }
    
        puntos_de_carga_tabla.put_item(
            Item = json.loads(json.dumps(item), parse_float=Decimal)
        )

df_mini.apply(import_punto_de_carga, axis=1)

0    None
0    None
1    None
1    None
1    None
1    None
2    None
2    None
2    None
2    None
2    None
3    None
3    None
4    None
4    None
5    None
5    None
5    None
5    None
6    None
7    None
8    None
9    None
9    None
dtype: object

In [27]:
import time

df_mini["ID_PuntoCarga"] = df_mini["index"]*1000 + df_mini["puntos_de_carga"]

estados_tabla = dynamodb.Table('Estado')

#Eliminamos las entradas anteriores
response = estados_tabla.scan()
for i in response['Items']:
    estados_tabla.delete_item(
        Key={
            'ID_Estado': i['ID_Estado']
        }
    )

def generate_status_pc(row):
    timestamp = int(time.time())
    item = {
            'ID_Estado': row["ID_PuntoCarga"] + int(timestamp) / (10 ** len(str(timestamp))),
            'ID_PuntoCarga': row["ID_PuntoCarga"],
            'estado': 0,
            'tiempo': timestamp,
            'matricula': "",
        }

    estados_tabla.put_item(
        Item = json.loads(json.dumps(item), parse_float=Decimal)
    )

df_mini.apply(generate_status_pc, axis=1)

0    None
0    None
1    None
1    None
1    None
1    None
2    None
2    None
2    None
2    None
2    None
3    None
3    None
4    None
4    None
5    None
5    None
5    None
5    None
6    None
7    None
8    None
9    None
9    None
dtype: object